<a href="https://colab.research.google.com/github/angiew-02/nhanes_inferential_2023/blob/main/nhanes_inferential.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
from scipy.stats import chi2_contingency
import numpy as np

Import Data

In [18]:
demo = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/DEMO_L.XPT', format='xport')
bp = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/BPXO_L.XPT', format='xport')
body = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/BMX_L.XPT', format='xport')
chol_total = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/TCHOL_L.XPT', format='xport')
glycohemo = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/GHB_L.XPT', format='xport')
crp = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/HSCRP_L.XPT', format='xport')
dm = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/DIQ_L.XPT', format='xport')
phy = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/PAQ_L.XPT', format='xport')
whd = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/WHQ_L.XPT', format='xport')

In [19]:
demographics = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/DEMO_L.XPT', format='xport')
demographics.DMDMARTZ

,DMDMARTZ
0,1.0
1,1.0
2,1.0
3,NaN
4,NaN
...,...
11928,NaN
11929,3.0
11930,1.0
11931,2.0


In [20]:
nhanes_data = demo.merge(bp, on='SEQN', how='left')\
                  .merge(body, on='SEQN', how='left')\
                  .merge(chol_total, on='SEQN', how='left')\
                  .merge(glycohemo, on='SEQN', how='left')\
                  .merge(crp, on='SEQN', how='left')\
                  .merge(dm, on='SEQN', how='left')\
                  .merge(phy, on='SEQN', how='left')\
                  .merge(whd, on='SEQN', how='left')

In [21]:
print(nhanes_data)

           SEQN  SDDSRVYR  RIDSTATR  RIAGENDR  RIDAGEYR  RIDAGEMN  RIDRETH1  \
0      130378.0      12.0       2.0       1.0      43.0       NaN       5.0   
1      130379.0      12.0       2.0       1.0      66.0       NaN       3.0   
2      130380.0      12.0       2.0       2.0      44.0       NaN       2.0   
3      130381.0      12.0       2.0       2.0       5.0       NaN       5.0   
4      130382.0      12.0       2.0       1.0       2.0       NaN       3.0   
...         ...       ...       ...       ...       ...       ...       ...   
11928  142306.0      12.0       2.0       1.0       9.0       NaN       2.0   
11929  142307.0      12.0       2.0       2.0      49.0       NaN       4.0   
11930  142308.0      12.0       2.0       1.0      50.0       NaN       2.0   
11931  142309.0      12.0       2.0       1.0      40.0       NaN       2.0   
11932  142310.0      12.0       2.0       2.0      80.0       NaN       3.0   

       RIDRETH3  RIDEXMON  RIDEXAGM  ...  PAD790U  

In [22]:
nhanes_data['DMDMARTZ'] = nhanes_data['DMDMARTZ'].apply(lambda x: 1 if x == 1 else 0)
nhanes_data['DMDEDUC2'] = nhanes_data['DMDEDUC2'].apply(lambda x: 1 if x in [4, 5] else 0)
nhanes_data['PAD680'].replace([7777, 9999], np.nan, inplace=True)
nhanes_data['WHD020'].replace([7777, 9999], np.nan, inplace=True)

<ipython-input-22-69fb298a54c5>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  nhanes_data['PAD680'].replace([7777, 9999], np.nan, inplace=True)
<ipython-input-22-69fb298a54c5>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace

Question 1: Is there an association between marital status and education level?

In [23]:
from scipy.stats import chi2_contingency
contingency_table = pd.crosstab(nhanes_data['DMDMARTZ'], nhanes_data['DMDEDUC2'])
chi2, p, dof, ex = chi2_contingency(contingency_table)
print(f"Chi2 Statistic: {chi2}, p-value: {p}")

### According to the statstic, there is no association between marital status and education level

Chi2 Statistic: 1677.006341355503, p-value: 0.0


Question 2: Is there a difference in the mean sedentary behavior time between those who are married and those who are not married?

In [12]:
from scipy.stats import ttest_ind
married = nhanes_data[nhanes_data['DMDMARTZ'] == 1]['PAD680'].dropna()
not_married = nhanes_data[nhanes_data['DMDMARTZ'] == 0]['PAD680'].dropna()
t_stat, p_val = ttest_ind(married, not_married)
print(f"T-Statistic: {t_stat}, p-value: {p_val}")

### There is a difference in sedentary behavior time between those who are married and those who are not married. Those who are married tend to spend less time being sedentary compared to those who are not married.

T-Statistic: -2.7114738497919184, p-value: 0.006712549062413583


Question 3: How do age and marital status affect systolic blood pressure?

In [13]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
model = ols('BPXOSY3 ~ RIDAGEYR + C(DMDMARTZ)', data=nhanes_data).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)

                   sum_sq      df            F    PR(>F)
C(DMDMARTZ)  1.737748e+03     1.0     6.860916  0.008828
RIDAGEYR     5.889160e+05     1.0  2325.138035  0.000000
Residual     1.893791e+06  7477.0          NaN       NaN


Question 4: Is there a correlation between self-reported weight and minutes of sedentary behavior?

In [16]:
subset_data = nhanes_data[['WHD020', 'PAD680']].dropna()
from scipy.stats import pearsonr
correlation, p_value = pearsonr(subset_data['WHD020'], subset_data['PAD680'])
print(f"Correlation: {correlation}, p-value: {p_value}")

### There is a small positive correlation between self-reported weight and minutes of sedentary behaviors

Correlation: 0.1046820984735302, p-value: 2.9649694069971987e-21
